In [109]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time

In [180]:
url = "https://www.boxofficemojo.com/year/"
year = "2022"

# response = requests.get(url+year)
# soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# OMDb API 
api_key = '6cf40218'
title_id = 'tt3896198'
http://www.omdbapi.com/?i=title_id&apikey=api_key

In [81]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# Create a WebDriver instance (e.g., for Firefox)
driver = webdriver.Firefox()

driver.get(url+year)

table = driver.find_element(By.XPATH, "//table")  # Adjust the XPath as needed to locate your table

driver.quit()

In [181]:
response = requests.get(url+year)
soup = BeautifulSoup(response.text, 'html.parser')

In [188]:
len(grosses)

400

In [191]:
# extracting the box office mojo IDs from each page
titles = soup.find_all('td', class_='a-text-left mojo-field-type-release mojo-cell-wide')
grosses = soup.find_all('td', class_='a-text-right mojo-field-type-money mojo-estimatable')

names = []
mojo_ids = []
dom_grosses = []

for title in titles:
    names.append(title.text)
    mojo_ids.append(title.find('a').get('href').split('/')[2])

for i in range(len(grosses)):
    if i % 2 == 0: # we want every other row 
        continue
    else:
        dom_grosses.append(int(grosses[i].text.replace('$','').replace(',','')))
    
df = pd.DataFrame({'name':names,'dom_gross':dom_grosses,'mojo_id':mojo_ids})

df.head()

,name,dom_gross,mojo_id
0,Top Gun: Maverick,718732821,rl2500036097
1,Black Panther: Wakanda Forever,453829060,rl3573908993
2,Doctor Strange in the Multiverse of Madness,411331607,rl3724903937
3,Avatar: The Way of Water,684075767,rl3372254721
4,Jurassic World Dominion,376851080,rl4043671041


In [59]:
## extract IMDb ID from BoxOfficeMojo Release Pages
url = "https://www.boxofficemojo.com/release/"
bom_id = "rl2500036097"

response = requests.get(url+bom_id)
soup = BeautifulSoup(response.text, 'html.parser')

# extract the IMDb Id
a = soup.find('div',id='title-summary-refiner').find('a').get('href').split('/')[2]

In [192]:
# function to scrape through each year of top 200 box office results
def imdb_ids(year):
    names = []
    years = []
    dom_grosses = []
    mojo_ids = []
    imdb_ids = []
    
    mojo_url = "https://www.boxofficemojo.com/year/"
    title_url = "https://www.boxofficemojo.com/release/"
    
#     year = start_year
    
#     if year <= end_year:
#         # box office mojo data
#         response = requests.get(mojo_url+str(year))
#         soup = BeautifulSoup(response.text, 'html.parser')
    
#         # extracting the box office mojo IDs from each page
#         titles = soup.find_all('td', class_='a-text-left mojo-field-type-release mojo-cell-wide')

#         for title in titles:
#             names.append(title.text)
#             mojo_ids.append(title.find('a').get('href').split('/')[2])
#             years.append(str(year))
        
#         # extract the domestic gross for each title
#         grosses = soup.find_all('td', class_='a-text-right mojo-field-type-money mojo-estimatable')

#         for gross in grosses:
#             dom_grosses.append(int(gross.text.replace('$','').replace(',','')))
        
#         # extracting imdb IDs
#         for i in range(len(mojo_ids)):
#             response = requests.get(title_url+mojo_ids[i])
#             soup = BeautifulSoup(response.text, 'html.parser')
        
#             try:
#                 imdb_ids.append(soup.find('div',id='title-summary-refiner').find('a').get('href').split('/')[2])
#                 print(f"{year} {(i+1)/len(mojo_ids)*100:.1f}% complete", end='\r')
#                 time.sleep(1)
#             except AttributeError:
#                 imdb_ids.append('NA')
#                 print(f"{year} {(i+1)/len(mojo_ids)*100:.1f}% complete", end='\r')
#                 time.sleep(1)
        
#         if year < end_year:
#             print(f"{year} complete, moving on to {year +1}")
#             year += 1
#         else:
#             print("All years complete")
#             year += 1
    
    # box office mojo data
    response = requests.get(mojo_url+str(year))
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # extracting the box office mojo IDs from each page
    titles = soup.find_all('td', class_='a-text-left mojo-field-type-release mojo-cell-wide')

    for title in titles:
        names.append(title.text)
        mojo_ids.append(title.find('a').get('href').split('/')[2])
        years.append(year)
        
    # extract the domestic gross for each title
    grosses = soup.find_all('td', class_='a-text-right mojo-field-type-money mojo-estimatable')

    for i in range(len(grosses)):
        if i % 2 == 0: # we want every other row 
            continue
        else:
            dom_grosses.append(int(grosses[i].text.replace('$','').replace(',','')))
        
    # extracting imdb IDs
    for i in range(len(mojo_ids)):
        response = requests.get(title_url+mojo_ids[i])
        soup = BeautifulSoup(response.text, 'html.parser')
        
        try:
            imdb_ids.append(soup.find('div',id='title-summary-refiner').find('a').get('href').split('/')[2])
            print(f"{(i+1)/len(mojo_ids)*100:.1f}% complete", end='\r')
            time.sleep(.5)
        except AttributeError:
            imdb_ids.append('NA')
            print(f"{(i+1)/len(mojo_ids)*100:.1f}% complete", end='\r')
            time.sleep(.5)
            
        
    df = pd.DataFrame({'name':names,'year':years,'domestic_gross':dom_grosses,'mojo_id':mojo_ids,'imdb_id':imdb_ids})
    
    return df    

In [193]:
start = time.time()

test = imdb_ids(2002)

stop = time.time()
stop-start

212.9302749633789

In [197]:
test.query("imdb_id == 'NA'")

,name,year,domestic_gross,mojo_id,imdb_id
169,Spider-Man/Men in Black IIDouble Bill,2002,6632970,rl695436801,NA


In [142]:
# df = imdb_ids('2022')

In [203]:
def get_data(start_year, end_year):
    '''
    iterate through years in a range and return a master dataframe
    '''
    # list of years
    years = [year for year in range(start_year, end_year + 1)]
    
    # master dataframe
    df = pd.DataFrame({
    'name':[],
    'year':[],
    'domestic_gross':[],
    'mojo_id':[],
    'imdb_id':[]
    })
    
    for year in years:
        data = imdb_ids(year)
        df = pd.concat([df, data], ignore_index=True)
        print(f"{year} complete", end='\r')
        
    return df

In [204]:
master_data = get_data(2002, 2022)

In [207]:
master_data.to_csv('imdb_ids.csv', index=False)

In [138]:
sample_list = list(range(10))
for i in range(len(sample_list)):
    print(f"{(i+1)/len(sample_list)*100:.0f}% complete", end='\r')
    time.sleep(1)